# Programación Entera: Heurísticas para el Problema de la Mochila

## Descripción

[TODO]

## Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
## Fecha

  * Mayo de 2018


## Contenidos
  * [Problema de la Mochila](#Problema-de-la-Mochila)
    * [Mochila Simple](#Mochila-Simple)
    * [Mochila Múltiple](#Mochila-Múltiple)
    * [Mochila Multidimensional](#Mochila-Multidimensional)

In [1]:
library(lpSolve)
library(magrittr)

In [2]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

## Funciones de Apoyo

In [3]:
dataset.folder <- "./data"

In [4]:
ReadDataSet <- function() {
    
}

In [5]:
Solve <- function(direction = "max", f.obj, f.con, f.rhs, f.dir = rep("<=", nrow(f.con)), ...) {
    lp(direction, f.obj, f.con, f.dir, f.rhs, ...)
}

## Problema de la Mochila

#### Descripción:
TODO

### Mochila Simple

#### Descripción:
TODO

In [6]:
ReadSimpleKnapsack <- function(file.name) {
    con <- 
        file.name %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit("[[:blank:]]") %>%
        unlist() %>%
        as.integer()
    close(con)
    list(
        n = raw[1],
        capacity = raw[2] ,
        value = raw[seq(4, length(raw), by = 3)],
        weight = raw[seq(5, length(raw), by = 3)]
    ) %>%
    return()
}

#### Modelo:
TODO

In [7]:
KnapsackSimpleExact <- function(p, ...) {
    return(Solve("max", p$value, matrix(p$weight, nrow = 1, byrow = TRUE), p$capacity, 
                 all.bin = TRUE, ...))    
}

#### Heurísticas:
TODO

In [8]:
KnapsackSimpleGreedy <- function(p, ...) {
    
}

### Ejemplos

In [9]:
gen100.1 <- ReadSimpleKnapsack("./data/knapsack/simple/gen100_1.txt")

In [10]:
a <- KnapsackSimpleExact(gen100.1)
matrix(a$solution, ncol = gen100.1$n, byrow = TRUE)
a$objval

1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


[1] 1753

In [11]:
gen100.2 <- ReadSimpleKnapsack("./data/knapsack/simple/gen100_2.txt")

In [12]:
a <- KnapsackSimpleExact(gen100.2)
matrix(a$solution, ncol = gen100.2$n, byrow = TRUE)
a$objval

0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0


[1] 1744

In [13]:
gen100.3 <- ReadSimpleKnapsack("./data/knapsack/simple/gen100_3.txt")

In [14]:
a <- KnapsackSimpleExact(gen100.3)
matrix(a$solution, ncol = gen100.3$n, byrow = TRUE)
a$objval

0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1


[1] 1898

In [15]:
gen100.4 <- ReadSimpleKnapsack("./data/knapsack/simple/gen100_4.txt")

In [16]:
a <- KnapsackSimpleExact(gen100.4)
matrix(a$solution, ncol = gen100.4$n, byrow = TRUE)
a$objval

0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


[1] 1825

In [17]:
gen100.5 <- ReadSimpleKnapsack("./data/knapsack/simple/gen100_5.txt")

In [18]:
a <- KnapsackSimpleExact(gen100.5)
matrix(a$solution, ncol = gen100.5$n, byrow = TRUE)
a$objval

0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0


[1] 1837

In [19]:
gen200.1 <- ReadSimpleKnapsack("./data/knapsack/simple/gen200_1.txt")

In [20]:
a <- KnapsackSimpleExact(gen200.1)
matrix(a$solution, ncol = gen200.1$n, byrow = TRUE)
a$objval

0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1


[1] 4965

In [21]:
gen200.2 <- ReadSimpleKnapsack("./data/knapsack/simple/gen200_2.txt")

In [22]:
a <- KnapsackSimpleExact(gen200.2)
matrix(a$solution, ncol = gen200.2$n, byrow = TRUE)
a$objval

0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0


[1] 4961

In [23]:
gen200.3 <- ReadSimpleKnapsack("./data/knapsack/simple/gen200_3.txt")

In [24]:
a <- KnapsackSimpleExact(gen200.3)
matrix(a$solution, ncol = gen200.3$n, byrow = TRUE)
a$objval

1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0


[1] 5232

In [25]:
gen200.4 <- ReadSimpleKnapsack("./data/knapsack/simple/gen200_4.txt")

In [26]:
a <- KnapsackSimpleExact(gen200.4)
matrix(a$solution, ncol = gen200.4$n, byrow = TRUE)
a$objval

0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0


[1] 4971

In [27]:
gen200.5 <- ReadSimpleKnapsack("./data/knapsack/simple/gen200_5.txt")

In [28]:
a <- KnapsackSimpleExact(gen200.5)
matrix(a$solution, ncol = gen200.5$n, byrow = TRUE)
a$objval

0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1


[1] 5031

In [29]:
gen300.1 <- ReadSimpleKnapsack("./data/knapsack/simple/gen300_1.txt")

In [30]:
a <- KnapsackSimpleExact(gen300.1)
matrix(a$solution, ncol = gen300.1$n, byrow = TRUE)
a$objval

1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,⋯,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0


[1] 11219

In [31]:
gen300.2 <- ReadSimpleKnapsack("./data/knapsack/simple/gen300_2.txt")

In [32]:
a <- KnapsackSimpleExact(gen300.2)
matrix(a$solution, ncol = gen300.2$n, byrow = TRUE)
a$objval

0,0,0,1,0,1,0,1,1,0,0,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,0,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,⋯,1,0,1,1,1,0,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,1,0,1,1,1,1,0,0,1,1,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1


[1] 11399

In [33]:
gen300.3 <- ReadSimpleKnapsack("./data/knapsack/simple/gen300_3.txt")

In [34]:
a <- KnapsackSimpleExact(gen300.3)
matrix(a$solution, ncol = gen300.3$n, byrow = TRUE)
a$objval

0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,1,1,⋯,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,1,1,0,1,1,1,0,1,0,0,0,1,1


[1] 11298

In [35]:
gen300.4 <- ReadSimpleKnapsack("./data/knapsack/simple/gen300_4.txt")

In [36]:
a <- KnapsackSimpleExact(gen300.4)
matrix(a$solution, ncol = gen300.4$n, byrow = TRUE)
a$objval

0,0,1,1,0,0,0,0,0,1,1,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,1,0,0,0,0,1,0,⋯,1,0,0,0,1,1,1,0,1,1,0,0,1,0,1,1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1


[1] 10985

In [37]:
gen300.5 <- ReadSimpleKnapsack("./data/knapsack/simple/gen300_5.txt")

In [38]:
a <- KnapsackSimpleExact(gen300.5)
matrix(a$solution, ncol = gen300.5$n, byrow = TRUE)
a$objval

1,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,1,1,⋯,0,1,1,0,1,1,1,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,1


[1] 11122

### Mochila Múltiple

#### Descripción:
TODO

#### Modelo:
TODO

In [39]:
KnapsackMultipleExact <- function(p, m, ...) {
    conditions <- matrix(rep(0,(m + p$n) * p$n * m), 
                         nrow = m + p$n, ncol = p$n * m, byrow = TRUE)
    
    
    
    for(i in 1:m) {
        conditions[i, (i - 1) * p$n + 1:p$n] <- p$weight
    }
    for(i in (m + 1):(p$n + m)) {
        conditions[i, seq(i-m, p$n*m, by = p$n)] <- 1
    }

    return(Solve("max", rep(p$value, m), conditions, c(rep(p$capacity, m), rep(1, p$n)),
                 all.bin = TRUE, ...))
}
gen10.1 <- ReadSimpleKnapsack("./data/knapsack/simple/gen10_1.txt")
m <- 3
a <- KnapsackMultipleExact(gen10.1, m)
matrix(a$solution, ncol = gen10.1$n, nrow = m, byrow = TRUE)
a$objval

0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,1
1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,1,0,0,0,0


[1] 2286

#### Heurísticas:
TODO

In [40]:
KnapsackMultipleGreedy <- function(object, capacity) {
    
}

### Ejemplos

### Mochila Multidimensional

#### Descripción:
TODO

In [41]:
ReadMultidimensionalKnapsack <- function(file.name) {
    con <- 
        file.name %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit("[[:blank:]]") %>%
        unlist() %>%
        as.double()
    close(con)
    list(n        = raw[1],
         m        = raw[2],
         objval   = raw[3],
         value    = raw[4:(raw[1]+ 3)],
         weight   = matrix(raw[(4 + raw[1]):(3 + raw[1] * (1 + raw[2]))], 
                           ncol = raw[1], byrow = TRUE),
         capacity = raw[(4 + raw[1] * (1 + raw[2]) ):length(raw)]
    ) %>%
    return()
}

#### Modelo:
TODO

In [42]:
KnapsackMultidimensionalExact <- function(p, ...) {    
    return(Solve("max", p$value, p$weight, p$capacity, all.bin = TRUE,...))
}

#### Heurísticas:
TODO

In [43]:
KnapsackMultidimensionalGreedy <- function(object, capacity) {
    
}

### Ejemplos

In [44]:
mknap1.2 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknap1_2.dat")

In [45]:
a <- KnapsackMultidimensionalExact(mknap1.2)
matrix(a$solution, ncol = mknap1.2$m, byrow = TRUE)
a$objval

0,1,0,1,1,0,0,1,0,1


[1] 8706.1

In [46]:
mknap1.3 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknap1_3.dat")

In [47]:
a <- KnapsackMultidimensionalExact(mknap1.3)
matrix(a$solution, ncol = mknap1.3$n, byrow = TRUE)
a$objval

1,1,0,1,0,1,1,0,1,1,0,0,0,1,1


[1] 4015

In [48]:
mknap1.4 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknap1_4.dat")

In [49]:
a <- KnapsackMultidimensionalExact(mknap1.4)
matrix(a$solution, ncol = mknap1.4$n, byrow = TRUE)
a$objval

1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1


[1] 6120

In [50]:
mknap1.6 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknap1_6.dat")

In [51]:
a <- KnapsackMultidimensionalExact(mknap1.6)
matrix(a$solution, ncol = mknap1.6$n, byrow = TRUE)
a$objval

1,1,0,1,0,1,0,1,1,0,1,0,1,0,1,1,1,1,1,1,0,0,1,0,1,0,1,1,1,0,1,1,0,1,1,1,1,1,1


[1] 10618

In [52]:
mknap1.7 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknap1_7.dat")

In [53]:
a <- KnapsackMultidimensionalExact(mknap1.7)
matrix(a$solution, ncol = mknap1.7$n, byrow = TRUE)
a$objval

0,0,0,1,0,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,0,0,1,0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1


[1] 16537

In [54]:
mknapcb1.1 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknapcb1_1.dat")

In [55]:
a <- KnapsackMultidimensionalExact(mknapcb1.1)
matrix(a$solution, ncol = mknapcb1.1$n, byrow = TRUE)
a$objval

0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,0,0,1,0


[1] 24381

In [56]:
mknapcb1.2 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknapcb1_2.dat")

In [57]:
a <- KnapsackMultidimensionalExact(mknapcb1.2)
matrix(a$solution, ncol = mknapcb1.2$n, byrow = TRUE)
a$objval

0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1


[1] 24274

In [58]:
mknapcb1.3 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknapcb1_3.dat")

In [59]:
a <- KnapsackMultidimensionalExact(mknapcb1.3)
matrix(a$solution, ncol = mknapcb1.3$n, byrow = TRUE)
a$objval

0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1


[1] 23551

In [60]:
mknapcb1.4 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknapcb1_4.dat")

In [61]:
a <- KnapsackMultidimensionalExact(mknapcb1.4)
matrix(a$solution, ncol = mknapcb1.4$n, byrow = TRUE)
a$objval

1,1,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0


[1] 23534

In [62]:
mknapcb1.5 <- ReadMultidimensionalKnapsack("./data/knapsack/multidimensional/mknapcb1_5.dat")

In [63]:
a <- KnapsackMultidimensionalExact(mknapcb1.5)
matrix(a$solution, ncol = mknapcb1.5$n, byrow = TRUE)
a$objval

0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,1,1


[1] 23991